[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/steps/step09.ipynb)

# 제1고지 미분 자동 계산

## 1단계 상자로서의 변수

## 2단계 변수를 낳는 함수

## 3단계 함수 연결

## 4단계 수치 미분

In [1]:
# def numerical_diff(f, x, eps=1e-4):
#     """

#       param:
#         f (Function): 미분의 대상이 되는 함수
#         x (Variable): 미분을 계산하는 변수
#         eps (float): 작은 값
#     """
#     x0 = Variable(x.data - eps)
#     x1 = Variable(x.data + eps)
#     y0 = f(x0)
#     y1 = f(x1)
#     return (y1.data - y0.data) / (2 * eps)

## 5단계 역전파 이론

## 6단계 수동 역전파

## 7단계 역전파 자동화

In [2]:
class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, input):
        x = input.data  # 데이터를 꺼낸다.
        y = self.forward(x)  # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y)  # Variable 형태로 되돌린다.
        output.set_creator(self)  # 출력 변수에 창조자를 설정한다.
        self.input = input  # 입력 변수를 기억(보관)한다.
        self.output = output  # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [3]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [4]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

## 8단계 재귀에서 반복문으로

In [5]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 1. 함수를 가져온다.
            x, y = f.input, f.output  # 2. 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)  # 3. backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.


## 9단계 함수를 더 편리하게

### 9.1 파이썬 함수로 이용하기

In [6]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

In [7]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [8]:
def square(x):
    return Square()(x)  # 한 줄로 작성

def exp(x):
    return Exp()(x)

In [9]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [10]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### 9.2 backward 메서드 간소화

In [11]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        ########################################
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        ########################################

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [12]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
# y.grad = np.array(1.0)  # 이 줄이 생략됨
y.backward()
print(x.grad)

3.297442541400256


### 9.3 ndarray만 취급하기

In [13]:
import numpy as np

class Variable:
    def __init__(self, data):
        ########################################
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        ########################################

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [14]:
x = Variable(np.array(1.0))

In [15]:
x = Variable(None)

In [16]:
x = Variable(1.0)

TypeError: <class 'float'>은(는) 지원하지 않습니다.

In [17]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [18]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [19]:
import numpy as np

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [20]:
import numpy as np
np.isscalar(np.float64(1.0))

True

In [21]:
np.isscalar(2.0)

True

In [22]:
np.isscalar(np.array(1.0))

False

In [23]:
np.isscalar(np.array([1, 2, 3]))

False

In [24]:
import numpy as np


class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        ########################################
        output = Variable(as_array(y))
        ########################################
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx